# Info
This notebook is about the process of dowloading the weather data from the internet.

First import the packages.

In [1]:
import os
import pandas as pd
import datetime
import csv

Set the download folder of the Chrome browser.

In [2]:
DOWNLOAD_FOLDER = 'D:\\letöltések\\weather_data'

Make sure you have an account for [amsz.hu](amsz.hu) and you are logged in, in the Chrome browser. Change the directories for your own system.

In [3]:
if len(os.listdir(DOWNLOAD_FOLDER)) == 0:
    os.chdir("C:\\Program Files (x86)\\Google\\Chrome\\Application\\") # Directory, where chrome.exe is located
    # Command to dowload the excel file for a speciffic day.
    base_command = '.\\chrome.exe "http://www.amsz.hu/ws/includes/export.php?id=4&save=xls&year={}&month={}&day={}"'
    base_command.format(2018, 5, 20)
    current_day = datetime.date(2008, 3, 2) # year, month, day # Day of the first record
    today = datetime.date.today()
    d_one_day = datetime.timedelta(days=1)
    while current_day < today:
        command = base_command.format(current_day.year, current_day.month, current_day.day)
        os.system(command)
        current_day += d_one_day
    # Change back the working directory.
    os.chdir('D:\\projects\\machine learning\\Deep Learning a gyakorlatban\\Deep-Learning-a-gyakorlatban-3-hazi')

The data is now on the local machine, but it is difficult to read old excel format (.xls) with pandas. So lets convert those to csv.

In [87]:
DATA_DIR = 'data'
CSV_DIR = os.path.join(DATA_DIR, 'amsz')
if not os.path.exists(CSV_DIR):
    os.mkdir(CSV_DIR)
    for file_name in os.listdir(DOWNLOAD_FOLDER):
        full_file_name = os.path.join(DOWNLOAD_FOLDER, file_name)
        with xlrd.open_workbook(full_file_name, encoding_override='cp1252') as workbook:
            sh = workbook.sheet_by_index(0)
            result_file_name = os.path.join(CSV_DIR, file_name[:-3] + 'csv')
            with open(result_file_name, 'w',  newline="") as f:
                c = csv.writer(f)
                for r in range(sh.nrows):
                    c.writerow(sh.row_values(r))

Now we can read the data, but there are many entries in multiple times a day. Let's take the average of those.

In [82]:
def getMean(ser, metric_length):
    # Filter empty enteries
    ser = ser[ser.map(lambda x:x[:-metric_length] != '---.-' and x[:-metric_length] != '----.-' and x[:-metric_length] != '---' and x[:-metric_length] != '--' and x[:-metric_length] != ' ')]
    return ser.map(lambda x: float(x[:-metric_length])).mean()

In [83]:
Temperature = []
Humidity =  []
Blast =  []
Rain =  []
WindSpeed =  []
WindDirection =  []
Day = []
for file_name in os.listdir(CSV_DIR):
    try:
        full_file_name = os.path.join(CSV_DIR, file_name)
        df = pd.read_csv(full_file_name, encoding='ansi', header=2).iloc[1:, :] # The first line after the header is empty
        temperature = getMean(df.Homérséklet, 2)
        humidity = getMean(df.Páratartalom, 1)
        blast = getMean(df.Légnyomás, 4)
        rain = getMean(df['Csapadék 24h'], 5)
        windSpeed = getMean(df['Szélsebesség'], 5)
        windDirection = getMean(df['Szélirány'], 1)
        day = datetime.datetime.strptime(df.Idopont.iloc[0][:10], '%Y-%m-%d').date()
        Temperature.append(temperature)
        Humidity.append(humidity)
        Blast.append(blast)
        Rain.append(rain)
        WindSpeed.append(windSpeed)
        WindDirection.append(windDirection)
        Day.append(day)
    except Exception as e:
        print(file_name)
        print(e)

Now that we have one entry for each day let's save the hole thing into a modern excel format (.xlsx). (.xlsx is smaller than .csv)

In [84]:
df = pd.DataFrame(data={'Temperature': Temperature,
                        'Humidity': Humidity,
                        'Blast': Blast,
                        'Rain': Rain,
                        'WindSpeed': WindSpeed,
                        'WindDirection':WindDirection},
                     index=Day)

In [90]:
df.to_excel(os.path.join(DATA_DIR, 'ansz.xlsx'))